In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
LANGCHAIN_TRACING_V2 = os.getenv("LANGCHAIN_TRACING_V2")
LANGCHAIN_ENDPOINT = os.getenv("LANGCHAIN_ENDPOINT")
LANGCHAIN_PROJECT_ID = os.getenv("LANGCHAIN_PROJECT_ID")

In [2]:
!pip install -qU langchain_community langchain_openai faiss-cpu

In [5]:
from langchain_community.document_loaders import WebBaseLoader

# 블로그 포스트 로드
loader = WebBaseLoader(
    "https://teddylee777.github.io/openai/openai-assistant-tutorial/", encoding="utf-8"
)


In [ ]:
from langchain.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 문서 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
docs = loader.load_and_split(text_splitter)

# 임베딩 정의
openai_embedding = OpenAIEmbeddings()

# 벡터DB 생성
db = FAISS.from_documents(docs, openai_embedding)

# retriever 생성
retriever = db.as_retriever()

# 문서 검색
query = "OpenAI Assistant API의 Functions 사용법에 대해 알려주세요."

4

In [7]:
relevant_docs = retriever.invoke(query)

# 검색된 문서의 개수 출력
len(relevant_docs)

4

In [9]:
from langchain_openai import ChatOpenAI


# ChatOpenAI 언어 모델을 초기화합니다. temperature는 0으로 설정합니다.
llm = ChatOpenAI(temperature=0, model="gpt-4o-mini")

In [ ]:
from langchain.retrievers.multi_query import MultiQueryRetriever

multiquery_retriever = MultiQueryRetriever.from_llm(  # MultiQueryRetriever를 언어 모델을 사용하여 초기화합니다.
    # 벡터 데이터베이스의 retriever와 언어 모델을 전달합니다.
    retriever=db.as_retriever(),
    llm=llm,
)

In [11]:
quary = "OpenAI Assistant API의 Functions 사용법에 대해 알려주세요."

relevant_docs = multiquery_retriever.invoke(query)

In [14]:
len(relevant_docs)

5

In [16]:
relevant_docs[0]

Document(id='fb48a187-ca16-4ba4-a01e-9d82447f6897', metadata={'source': 'https://teddylee777.github.io/openai/openai-assistant-tutorial/', 'title': '[Assistants API] Code Interpreter, Retrieval, Functions 활용법 - 테디노트', 'description': 'OpenAI의 새로운 Assistants API는 상태 유지 대화와 강력한 도구 접근성을 제공합니다. OpenAI Assistants API 튜토리얼: Code Interpreter, Retrieval, Functions 활용법에 대하여 알아보겠습니다.', 'language': 'ko'}, page_content='OpenAI의 새로운 Assistants API는 대화와 더불어 강력한 도구 접근성을 제공합니다. 본 튜토리얼은 OpenAI Assistants API를 활용하는 내용을 다룹니다. 특히, Assistant API 가 제공하는 도구인 Code Interpreter, Retrieval, Functions 를 활용하는 방법에 대해 다룹니다. 이와 더불어 파일을 업로드 하는 내용과 사용자의 피드백을 제출하는 내용도 튜토리얼 말미에 포함하고 있습니다.\n\n\n\n주요내용')